<h2>Simple MNIST convnet - SR Nov21</h2>c.f. https://keras.io/examples/vision/mnist_convnet/

In [7]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [10]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

tf.test.is_gpu_available()


2022-12-07 22:01:36.205112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 22:01:36.205374: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False

In [11]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [12]:
model = keras.Sequential(
    [   layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [13]:
batch_size = 128
epochs = 15
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 19s 42ms/step - loss: 0.3665 - accuracy: 0.8894 - val_loss: 0.0889 - val_accuracy: 0.9755
Epoch 2/15
422/422 [==============================] - 17s 41ms/step - loss: 0.1120 - accuracy: 0.9659 - val_loss: 0.0582 - val_accuracy: 0.9847
Epoch 3/15
422/422 [==============================] - 17s 41ms/step - loss: 0.0844 - accuracy: 0.9738 - val_loss: 0.0465 - val_accuracy: 0.9882
Epoch 4/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0710 - accuracy: 0.9782 - val_loss: 0.0452 - val_accuracy: 0.9885
Epoch 5/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0621 - accuracy: 0.9811 - val_loss: 0.0404 - val_accuracy: 0.9888
Epoch 6/15
422/422 [==============================] - 18s 44ms/step - loss: 0.0559 - accuracy: 0.9826 - val_loss: 0.0347 - val_accuracy: 0.9907
Epoch 7/15
422/422 [==============================] - 18s 43ms/step - loss: 0.0508 - accuracy: 0.9839 - val_loss: 0.0340 - val_accuracy:

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02683436870574951
Test accuracy: 0.9909999966621399


OK, now let us play with the network and supply it with our own data...

In [15]:
x_test.shape, y_test.shape

((10000, 28, 28, 1), (10000, 10))

In [31]:
model.predict(x_test[0].reshape(1,28,28,1))

array([[1.6251480e-09, 4.3673240e-10, 2.1971598e-06, 1.3731418e-06,
        4.4243136e-13, 2.4493607e-10, 1.9594806e-16, 9.9999630e-01,
        5.6071374e-09, 1.2497229e-07]], dtype=float32)

In [33]:
test=1-np.fromfile('test-image2.data',dtype=np.ubyte)/255
model.predict(x_test[0].reshape(1,28,28,1))

array([[1.6251480e-09, 4.3673240e-10, 2.1971598e-06, 1.3731418e-06,
        4.4243136e-13, 2.4493607e-10, 1.9594806e-16, 9.9999630e-01,
        5.6071374e-09, 1.2497229e-07]], dtype=float32)